In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn plotly scikit-learn streamlit pyngrok gradio --quiet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import re
import gradio as gr
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

In [ ]:
# Create sample job skills dataset
def create_sample_data():
    # Sample job roles and their required skills
    job_roles = {
        'Data Scientist': ['python', 'machine learning', 'statistics', 'sql', 'data visualization',
                          'deep learning', 'pandas', 'numpy', 'tensorflow', 'pytorch'],
        'Data Analyst': ['sql', 'excel', 'tableau', 'python', 'statistics', 'data visualization',
                        'power bi', 'r', 'data cleaning', 'reporting'],
        'Machine Learning Engineer': ['python', 'machine learning', 'deep learning', 'tensorflow',
                                     'pytorch', 'docker', 'aws', 'mlops', 'kubernetes', 'ci/cd'],
        'Business Analyst': ['sql', 'excel', 'powerpoint', 'tableau', 'requirements gathering',
                           'stakeholder management', 'uml', 'agile', 'jira', 'power bi'],
        'Software Engineer': ['java', 'python', 'javascript', 'react', 'node.js', 'docker',
                            'aws', 'git', 'ci/cd', 'system design'],
        'DevOps Engineer': ['aws', 'docker', 'kubernetes', 'jenkins', 'terraform', 'linux',
                          'python', 'ci/cd', 'monitoring', 'ansible'],
        'Frontend Developer': ['javascript', 'react', 'html', 'css', 'typescript', 'vue.js',
                             'angular', 'responsive design', 'webpack', 'git'],
        'Backend Developer': ['python', 'java', 'node.js', 'sql', 'mongodb', 'docker',
                            'aws', 'rest api', 'microservices', 'spring boot']
    }

    # Sample user profiles with their skills
    user_profiles = [
        {'name': 'User1', 'skills': ['python', 'sql', 'excel', 'statistics', 'data visualization']},
        {'name': 'User2', 'skills': ['java', 'javascript', 'html', 'css', 'git']},
        {'name': 'User3', 'skills': ['python', 'machine learning', 'pandas', 'numpy', 'sql']},
        {'name': 'User4', 'skills': ['aws', 'docker', 'linux', 'python', 'git']},
        {'name': 'User5', 'skills': ['sql', 'excel', 'powerpoint', 'tableau', 'statistics']}
    ]

    # Create datasets
    jobs_data = []
    for role, skills in job_roles.items():
        jobs_data.append({
            'job_role': role,
            'required_skills': skills,
            'skills_text': ' '.join(skills)
        })

    users_data = []
    for user in user_profiles:
        users_data.append({
            'user_name': user['name'],
            'user_skills': user['skills'],
            'skills_text': ' '.join(user['skills'])
        })

    return pd.DataFrame(jobs_data), pd.DataFrame(users_data)

# Create the datasets
jobs_df, users_df = create_sample_data()

print("Sample datasets created successfully!")
print("\nJob Roles:")
print(jobs_df[['job_role', 'required_skills']])
print("\nUser Profiles:")
print(users_df[['user_name', 'user_skills']])

In [ ]:
class SkillGapAnalyzer:
    def __init__(self, jobs_df, users_df):
        self.jobs_df = jobs_df
        self.users_df = users_df
        self.vectorizer = TfidfVectorizer()

    def analyze_skill_gap(self, user_skills, target_job):
        """Analyze skill gap between user skills and target job requirements"""
        # Get job requirements
        job_row = self.jobs_df[self.jobs_df['job_role'] == target_job]
        if job_row.empty:
            return None, None, None

        required_skills = job_row['required_skills'].iloc[0]
        job_skills_text = job_row['skills_text'].iloc[0]

        # Prepare texts for similarity calculation
        texts = [job_skills_text, ' '.join(user_skills)]

        # Calculate TF-IDF vectors
        tfidf_matrix = self.vectorizer.fit_transform(texts)

        # Calculate cosine similarity
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

        # Find matching and missing skills
        matching_skills = set(user_skills) & set(required_skills)
        missing_skills = set(required_skills) - set(user_skills)

        return matching_skills, missing_skills, similarity

    def get_recommendations(self, user_skills):
        """Get job recommendations based on user skills"""
        recommendations = []

        for _, job in self.jobs_df.iterrows():
            matching_skills, missing_skills, similarity = self.analyze_skill_gap(
                user_skills, job['job_role']
            )

            if similarity is not None:
                recommendations.append({
                    'job_role': job['job_role'],
                    'match_score': round(similarity * 100, 2),
                    'matching_skills': list(matching_skills),
                    'missing_skills': list(missing_skills),
                    'matching_count': len(matching_skills),
                    'missing_count': len(missing_skills)
                })

        # Sort by match score
        recommendations.sort(key=lambda x: x['match_score'], reverse=True)
        return recommendations

    def visualize_skill_gap(self, user_skills, target_job):
        """Create visualization for skill gap analysis"""
        matching_skills, missing_skills, similarity = self.analyze_skill_gap(
            user_skills, target_job
        )

        if similarity is None:
            return None

        # Create skill comparison chart
        fig = go.Figure()

        fig.add_trace(go.Bar(
            y=['Match Score'],
            x=[similarity * 100],
            name='Match Score',
            orientation='h',
            marker_color='lightblue'
        ))

        fig.update_layout(
            title=f'Skill Match for {target_job}',
            xaxis_title='Match Percentage (%)',
            showlegend=False
        )

        return fig

    def generate_learning_path(self, missing_skills):
        """Generate learning path recommendations for missing skills"""
        skill_resources = {
            'python': ['Python for Everybody Coursera', 'Codecademy Python', 'Real Python Tutorials'],
            'machine learning': ['Coursera ML by Andrew Ng', 'Fast.ai', 'Kaggle Learn'],
            'sql': ['SQLZoo', 'Mode Analytics SQL Tutorial', 'W3Schools SQL'],
            'data visualization': ['Tableau Training', 'Plotly Documentation', 'Seaborn Tutorial'],
            'deep learning': ['Deep Learning Specialization Coursera', 'PyTorch Tutorials'],
            'aws': ['AWS Training & Certification', 'AWS Workshops', 'A Cloud Guru'],
            'docker': ['Docker Getting Started', 'Docker Curriculum', 'Play with Docker'],
            'javascript': ['JavaScript.info', 'MDN Web Docs', 'FreeCodeCamp JavaScript'],
            'react': ['React Official Tutorial', 'Fullstack Open', 'React Patterns'],
            'java': ['Java MOOC', 'Java Programming on Coursera', 'Baeldung Java'],
            'excel': ['Microsoft Excel Tutorial', 'Excel Easy', 'Chandoo.org'],
            'tableau': ['Tableau Training Videos', 'Tableau Public Gallery', 'Tableau Tutorials']
        }

        learning_path = {}
        for skill in missing_skills:
            if skill in skill_resources:
                learning_path[skill] = skill_resources[skill]
            else:
                learning_path[skill] = [f"Online courses for {skill}", f"Practice projects for {skill}"]

        return learning_path

# Initialize analyzer
analyzer = SkillGapAnalyzer(jobs_df, users_df)

In [13]:
# ===== REPLACE YOUR OLD INTERFACE CODE WITH THIS FIXED VERSION =====

def get_specific_analysis_html(user_name, user_skills, target_job):
    matching_skills, missing_skills, similarity = analyzer.analyze_skill_gap(user_skills, target_job)

    if similarity is None:
        return f"<div style='color: red; padding: 20px;'>Job role '{target_job}' not found in our database.</div>"

    learning_path = analyzer.generate_learning_path(missing_skills)

    # Create progress bar
    progress_width = similarity * 100
    progress_color = "#27ae60" if progress_width > 70 else "#f39c12" if progress_width > 50 else "#e74c3c"

    html = f"""
    <div style='font-family: Arial, sans-serif;'>
        <h2 style='color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 10px;'>📊 Skill Gap Analysis: {target_job}</h2>
        <p style='color: #34495e;'><strong>Analyzing for:</strong> {user_name}</p>
        <p style='color: #34495e;'><strong>Your Skills:</strong> {', '.join(user_skills)}</p>

        <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 10px; margin: 20px 0; text-align: center;'>
            <h3 style='margin: 0 0 10px 0; color: white;'>Overall Match Score</h3>
            <div style='background: rgba(255,255,255,0.2); border-radius: 20px; height: 30px; margin: 15px 0; overflow: hidden;'>
                <div style='background-color: {progress_color}; height: 100%; width: {progress_width}%; border-radius: 20px; display: flex; align-items: center; justify-content: center; color: white; font-weight: bold;'>
                    {progress_width:.1f}%
                </div>
            </div>
            <p style='margin: 0; font-size: 14px; color: white;'>You have {len(matching_skills)} out of {len(matching_skills) + len(missing_skills)} required skills</p>
        </div>

        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin: 20px 0;'>
            <div style='background-color: #e8f6f3; padding: 20px; border-radius: 10px; border-left: 5px solid #27ae60;'>
                <h4 style='color: #27ae60; margin: 0 0 15px 0;'>✅ Your Matching Skills ({len(matching_skills)})</h4>
                <div style='display: flex; flex-wrap: wrap; gap: 8px;'>
                    {''.join([f'<span style="background-color: #27ae60; color: white; padding: 5px 12px; border-radius: 15px; font-size: 12px; font-weight: bold;">{skill}</span>' for skill in matching_skills]) if matching_skills else '<p style="color: #666; font-style: italic; margin: 0;">No matching skills found</p>'}
                </div>
            </div>

            <div style='background-color: #fdebd0; padding: 20px; border-radius: 10px; border-left: 5px solid #f39c12;'>
                <h4 style='color: #f39c12; margin: 0 0 15px 0;'>📚 Skills to Learn ({len(missing_skills)})</h4>
                <div style='display: flex; flex-wrap: wrap; gap: 8px;'>
                    {''.join([f'<span style="background-color: #f39c12; color: white; padding: 5px 12px; border-radius: 15px; font-size: 12px; font-weight: bold;">{skill}</span>' for skill in missing_skills]) if missing_skills else '<p style="color: #666; font-style: italic; margin: 0;">Perfect match! No skills to learn</p>'}
                </div>
            </div>
        </div>
    """

    if missing_skills:
        html += """
        <div style='background-color: #f8f9fa; padding: 25px; border-radius: 10px; margin: 20px 0; border: 1px solid #e9ecef;'>
            <h3 style='color: #2c3e50; margin: 0 0 20px 0;'>🎯 Recommended Learning Path</h3>
        """

        for skill, resources in learning_path.items():
            html += f"""
            <div style='margin: 15px 0; padding: 15px; background: white; border-radius: 8px; border-left: 4px solid #3498db; box-shadow: 0 2px 4px rgba(0,0,0,0.1);'>
                <h4 style='color: #2c3e50; margin: 0 0 10px 0;'>📖 {skill.title()}</h4>
                <ul style='margin: 0; padding-left: 20px; color: #34495e;'>
            """
            for resource in resources:
                html += f"<li style='margin: 5px 0; color: #34495e;'>{resource}</li>"
            html += "</ul></div>"

        html += "</div>"

    html += "</div>"
    return html

def get_job_recommendations_html(user_name, user_skills):
    recommendations = analyzer.get_recommendations(user_skills)

    if not recommendations:
        return "<div style='color: red; padding: 20px;'>No recommendations found. Please check your skills input.</div>"

    html = f"""
    <div style='font-family: Arial, sans-serif;'>
        <h2 style='color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 10px;'>🎯 Job Recommendations for {user_name}</h2>
        <p style='color: #34495e;'><strong>Your Skills:</strong> {', '.join(user_skills)}</p>
    """

    for i, rec in enumerate(recommendations[:6], 1):
        match_color = "#27ae60" if rec['match_score'] > 70 else "#f39c12" if rec['match_score'] > 50 else "#e74c3c"

        html += f"""
        <div style='border: 1px solid #bdc3c7; border-radius: 10px; padding: 20px; margin: 15px 0; background: white; box-shadow: 0 2px 5px rgba(0,0,0,0.1);'>
            <div style='display: flex; justify-content: space-between; align-items: center;'>
                <h3 style='color: #2c3e50; margin: 0;'>{i}. {rec['job_role']}</h3>
                <span style='background-color: {match_color}; color: white; padding: 5px 15px; border-radius: 20px; font-weight: bold;'>
                    {rec['match_score']}% Match
                </span>
            </div>

            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-top: 15px;'>
                <div style='background-color: #d5f4e6; padding: 15px; border-radius: 8px;'>
                    <h4 style='color: #27ae60; margin: 0 0 10px 0;'>✅ Matching Skills ({rec['matching_count']})</h4>
                    <p style='margin: 0; color: #34495e;'>{', '.join(rec['matching_skills']) if rec['matching_skills'] else 'None'}</p>
                </div>
                <div style='background-color: #fadbd8; padding: 15px; border-radius: 8px;'>
                    <h4 style='color: #e74c3c; margin: 0 0 10px 0;'>📚 Skills to Learn ({rec['missing_count']})</h4>
                    <p style='margin: 0; color: #34495e;'>{', '.join(rec['missing_skills']) if rec['missing_skills'] else 'Perfect match!'}</p>
                </div>
            </div>
        </div>
        """

    html += "</div>"
    return html

def create_fixed_interface():
    with gr.Blocks(theme=gr.themes.Soft(), title="🎯 Job Skill Gap Analyzer") as demo:
        gr.Markdown("""
        # 🎯 Job Skill Gap Analyzer
        *Analyze the gap between your current skills and your dream job requirements!*
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 👤 Your Profile")

                user_name = gr.Textbox(
                    label="Your Name",
                    placeholder="Enter your name...",
                    value="John Doe"
                )

                user_skills = gr.Textbox(
                    label="Your Skills",
                    placeholder="Enter comma-separated skills: python, sql, machine learning...",
                    value="python, sql, statistics, excel",
                    lines=3
                )

                analysis_type = gr.Radio(
                    choices=["🎯 Job Recommendations", "📊 Specific Job Analysis"],
                    label="Analysis Type",
                    value="🎯 Job Recommendations"
                )

                target_job = gr.Dropdown(
                    choices=list(jobs_df['job_role'].values),
                    label="Target Job Role",
                    value="Data Scientist",
                    visible=False
                )

                with gr.Row():
                    analyze_btn = gr.Button("Analyze Skill Gap 🚀", variant="primary", scale=2)
                    clear_btn = gr.Button("Clear", variant="secondary", scale=1)

            with gr.Column(scale=2):
                gr.Markdown("### 📊 Analysis Results")
                output = gr.HTML(
                    value="<div style='text-align: center; padding: 40px; color: #666;'><h3>Your analysis results will appear here</h3><p>Enter your skills and click 'Analyze Skill Gap' to get started!</p></div>"
                )

        # Function to update target job visibility
        def update_visibility(analysis_type):
            return gr.Dropdown(visible=("Specific Job Analysis" in analysis_type))

        analysis_type.change(
            update_visibility,
            inputs=[analysis_type],
            outputs=[target_job]
        )

        # Analysis function
        def analyze_skills(user_name, user_skills_text, analysis_type, target_job=None):
            user_skills = [skill.strip().lower() for skill in user_skills_text.split(',') if skill.strip()]

            if analysis_type == "🎯 Job Recommendations":
                return get_job_recommendations_html(user_name, user_skills)
            else:
                if not target_job:
                    return "<div style='color: red; padding: 20px;'>Please select a target job role for specific analysis.</div>"
                return get_specific_analysis_html(user_name, user_skills, target_job)

        # Connect buttons
        analyze_btn.click(
            fn=analyze_skills,
            inputs=[user_name, user_skills, analysis_type, target_job],
            outputs=[output]
        )

        def clear_inputs():
            return "", "", "🎯 Job Recommendations", "Data Scientist"

        clear_btn.click(
            fn=clear_inputs,
            outputs=[user_name, user_skills, analysis_type, target_job]
        )

        # Examples
        gr.Markdown("### 🚀 Quick Examples")
        gr.Examples(
            examples=[
                ["Data Enthusiast", "python, sql, excel, statistics, data visualization", "🎯 Job Recommendations", None],
                ["Tech Student", "python, machine learning, pandas, numpy, git", "📊 Specific Job Analysis", "Data Scientist"],
                ["IT Professional", "java, python, docker, aws, git, linux", "🎯 Job Recommendations", None],
                ["Analytics Beginner", "excel, powerpoint, sql, tableau, statistics", "📊 Specific Job Analysis", "Data Analyst"]
            ],
            inputs=[user_name, user_skills, analysis_type, target_job],
            label="Click any example to load it"
        )

    return demo

# Create and launch the fixed interface
print("🚀 Creating the fixed interface with better colors...")
fixed_demo = create_fixed_interface()
print("🎯 Launching the interface...")
fixed_demo.launch(share=True, debug=False)

🚀 Creating the fixed interface with better colors...
🎯 Launching the interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d3739060067e8eeb4a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
